## Lab assignment 02

### Neural Machine Translation in the wild
In the third homework you are supposed to get the best translation you can for the EN-RU translation task.

Basic approach using RNNs as encoder and decoder is implemented for you. 

Your ultimate task is to use the techniques we've covered, e.g.

* Optimization enhancements (e.g. learning rate decay)

* CNN encoder (with or without positional encoding)

* attention/self-attention mechanism

* pretraining the language model

* [Byte Pair Encoding](https://github.com/rsennrich/subword-nmt)

* or just fine-tunning BERT ;)

to improve the translation quality. 

__Please use at least three different approaches/models and compare them (translation quality/complexity/training and evaluation time).__

Write down some summary on your experiments and illustrate it with convergence plots/metrics and your thoughts. Just like you would approach a real problem.

In [1]:
#!pip install -U torch==1.10 torchtext torchvision

In [2]:
from tqdm.notebook import tqdm

In [3]:
import pandas as pd
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

In [4]:
import torch
torch.__version__

'1.11.0+cu113'

In [5]:
# You might need to install the libraries below. Do it in the desired environment
# if you are working locally.

#! pip  install subword-nmt
#! pip install nltk
#! pip install torchtext

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim

#import torchtext
#from torchtext.datasets import TranslationDataset, Multi30k
#from torchtext.data import Field, BucketIterator
#from torchtext.datasets import TranslationDataset, Multi30k
#from torchtext.data import Field, BucketIterator

#import spacy

import random
import math
import time

import matplotlib
matplotlib.rcParams.update({'figure.figsize': (16, 12), 'font.size': 14})
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import clear_output

#from nltk.tokenize import WordPunctTokenizer
#from subword_nmt.learn_bpe import learn_bpe
#from subword_nmt.apply_bpe import BPE


In [7]:
#from nltk.translate.bleu_score import corpus_bleu


In [8]:
import numpy as np

In [9]:
df = pd.read_csv(r'data\recsys-in-practice\train_joke_df.csv')

df.head(2)

,UID,JID,Rating
0,18029,6,-1.26
1,3298,64,-4.17


In [10]:
#rating = df['Rating'].values
#print(np.min(rating), np.max(np.abs(rating)), np.max(np.abs(rating)))

#rating_norm = (rating - np.min(rating)) / (np.max(rating) - np.min(rating))
#rating_norm = rating / np.max(np.abs(rating))
#print(np.min(rating_norm), np.max(rating_norm))

#df['Rating'] = rating_norm

In [11]:
# сделаем сортировку и перепишем index
df = df.sort_values(by=['UID', 'JID'])
df = df.reset_index(drop=True)
df.head(2)

,UID,JID,Rating
0,1,1,-7.82
1,1,2,8.79


In [12]:
df_gr = df.groupby(by='UID', as_index=False).agg({'JID':list, 'Rating':list})
df_gr

,UID,JID,Rating
0,1,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15...","[-7.82, 8.79, -9.66, -8.16, -7.52, -8.5, -9.85..."
1,2,"[1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 15, 1...","[4.08, -0.29, 6.36, 4.37, -9.66, -0.73, -5.34,..."
2,3,"[5, 6, 7, 8, 11, 12, 13, 15, 16, 17, 18, 19, 2...","[9.03, 9.27, 9.03, 9.27, 7.33, 7.57, 9.37, -6...."
3,4,"[2, 5, 7, 8, 10, 11, 13, 14, 15, 16, 17, 18, 1...","[8.35, 1.8, -2.82, 6.21, 1.84, 7.33, 6.31, 8.1..."
4,5,"[1, 2, 3, 4, 7, 9, 11, 12, 13, 14, 15, 16, 18,...","[8.5, 4.61, -4.17, -5.39, 7.04, -0.44, 8.25, 6..."
...,...,...,...
24978,24979,"[1, 4, 6, 7, 8, 11, 12, 13, 14, 15, 16, 17, 18...","[0.44, 2.33, 6.75, -8.79, -0.53, -0.53, 5.63, ..."
24979,24980,"[2, 3, 4, 5, 6, 7, 8, 9, 11, 13, 14, 15, 16, 1...","[-8.16, 8.59, 9.08, 0.87, -8.93, -3.5, 5.78, -..."
24980,24981,"[5, 8, 13, 14, 17, 18, 20, 26, 28, 32, 35, 36,...","[-7.77, -6.75, -6.46, -1.65, -6.99, 7.23, -6.9..."
24981,24982,"[5, 7, 8, 13, 15, 16, 17, 18, 19, 20, 21, 28, ...","[-9.71, 4.56, -8.3, -9.47, 3.45, -0.92, -4.51,..."


In [13]:
len(df_gr.values[0][1])

62

In [14]:
len(df_gr.values[0][2])

62

In [15]:
max_joke_id = np.max(np.unique(df['JID']))
sos_token = max_joke_id + 1
eos_token = max_joke_id + 2
pad_token = max_joke_id + 3
mask_token = max_joke_id + 4
max_joke_id, sos_token, eos_token, pad_token, mask_token

(100, 101, 102, 103, 104)

In [16]:
max_len = np.max(df_gr['JID'].apply(len))
min_len = np.min(df_gr['JID'].apply(len))
min_len, max_len

(18, 95)

In [17]:
seq_len = max_len + 2

In [18]:
trg_seq_len = 5

In [19]:
for row in df_gr.values:
    
    ext_len = max_len - len(row[1])
    
    row[1].insert(0, sos_token)
    row[1].extend([pad_token] * ext_len)
    row[1].append(eos_token)
    
    
    row[2].insert(0, 0)
    row[2].extend([0] * ext_len)
    row[2].append(0)
    
max_len = np.max(df_gr['JID'].apply(len))
min_len = np.min(df_gr['JID'].apply(len))
print(min_len, max_len)
df_gr

97 97


,UID,JID,Rating
0,1,"[101, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 1...","[0, -7.82, 8.79, -9.66, -8.16, -7.52, -8.5, -9..."
1,2,"[101, 1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, ...","[0, 4.08, -0.29, 6.36, 4.37, -9.66, -0.73, -5...."
2,3,"[101, 5, 6, 7, 8, 11, 12, 13, 15, 16, 17, 18, ...","[0, 9.03, 9.27, 9.03, 9.27, 7.33, 7.57, 9.37, ..."
3,4,"[101, 2, 5, 7, 8, 10, 11, 13, 14, 15, 16, 17, ...","[0, 8.35, 1.8, -2.82, 6.21, 1.84, 7.33, 6.31, ..."
4,5,"[101, 1, 2, 3, 4, 7, 9, 11, 12, 13, 14, 15, 16...","[0, 8.5, 4.61, -4.17, -5.39, 7.04, -0.44, 8.25..."
...,...,...,...
24978,24979,"[101, 1, 4, 6, 7, 8, 11, 12, 13, 14, 15, 16, 1...","[0, 0.44, 2.33, 6.75, -8.79, -0.53, -0.53, 5.6..."
24979,24980,"[101, 2, 3, 4, 5, 6, 7, 8, 9, 11, 13, 14, 15, ...","[0, -8.16, 8.59, 9.08, 0.87, -8.93, -3.5, 5.78..."
24980,24981,"[101, 5, 8, 13, 14, 17, 18, 20, 26, 28, 32, 35...","[0, -7.77, -6.75, -6.46, -1.65, -6.99, 7.23, -..."
24981,24982,"[101, 5, 7, 8, 13, 15, 16, 17, 18, 19, 20, 21,...","[0, -9.71, 4.56, -8.3, -9.47, 3.45, -0.92, -4...."


In [20]:
df_gr.values[0][1][-10:]

[103, 103, 103, 103, 103, 103, 103, 103, 103, 102]

In [21]:
df_gr.values[0][2][-10:]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [22]:
tokens = df_gr.values[0][1]


count = np.random.randint(1, 5)
indices = np.random.choice(np.where(tokens <= max_joke_id)[0], count)

x = torch.tensor(tokens)
x[indices] = mask_token
x

tensor([101,   1,   2,   3,   4,   5,   6,   7,   8,   9, 104,  12,  13,  14,
         15,  16,  17,  18,  20,  21,  22,  23,  24,  26,  27,  28,  29,  30,
         32,  33,  35,  36,  37, 104,  41,  42,  43,  44,  45,  47,  48,  50,
         51,  52,  53,  54,  55,  56,  57, 104,  59,  61,  62,  63,  64,  66,
         67,  68,  69,  70,  78,  82,  97, 103, 103, 103, 103, 103, 103, 103,
        103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103,
        103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 102])

In [23]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [24]:
class FeaturesDataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.df = df
            
    def __getitem__(self, index: int):    
        tokens = self.df.values[index][1]
        ratings = self.df.values[index][2]

        count = trg_seq_len#np.random.randint(1, 5)
        indices = np.array(np.random.choice(np.where(tokens <= max_joke_id)[0], count), dtype=int)

        #todo
        indices = [10, 20, 30, 40, 50]
        #print(indices)
        tokens_src = torch.tensor(tokens, device=device)
        tokens_trg = tokens_src[indices].detach().clone().to(device)
        tokens_src[indices] = mask_token
        
        ratings_src = torch.tensor(ratings, device=device)
        ratings_trg = ratings_src[indices].detach().clone().to(device)
        ratings_src[indices] = 0

        return tokens_src, tokens_trg, ratings_src, ratings_trg
    
    def __len__(self):
        return len(self.df)

In [25]:
ds = FeaturesDataset(df_gr)

In [26]:
ds[0]

(tensor([101,   1,   2,   3,   4,   5,   6,   7,   8,   9, 104,  12,  13,  14,
          15,  16,  17,  18,  20,  21, 104,  23,  24,  26,  27,  28,  29,  30,
          32,  33, 104,  36,  37,  38,  41,  42,  43,  44,  45,  47, 104,  50,
          51,  52,  53,  54,  55,  56,  57,  58, 104,  61,  62,  63,  64,  66,
          67,  68,  69,  70,  78,  82,  97, 103, 103, 103, 103, 103, 103, 103,
         103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103,
         103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 102],
        device='cuda:0'),
 tensor([10, 22, 35, 48, 59], device='cuda:0'),
 tensor([ 0.0000, -7.8200,  8.7900, -9.6600, -8.1600, -7.5200, -8.5000, -9.8500,
          4.1700, -8.9800,  0.0000, -6.7500, -7.1800,  8.4500, -7.1800, -7.5200,
         -7.4300, -9.8100, -9.8500, -9.3700,  0.0000, -4.3700, -9.8100,  1.1200,
          7.8200,  2.8600,  9.1300, -7.4300, -4.0800, -9.0800,  0.0000,  4.9500,
         -9.1700, -8.4000, -8.1100, -9.1300, -9.0300,

### Main part
__Here comes the preprocessing. Do not hesitate to use BPE or more complex preprocessing ;)__

### Model side
__Here comes simple pipeline of NMT model learning. It almost copies the week03 practice__

In [27]:
batch_size=7

for tokens_src, tokens_trg, ratings_src, ratings_trg in torch.utils.data.DataLoader(ds, batch_size= batch_size):
    #print(x, y)
    break
    
tokens_src.shape, tokens_trg.shape, ratings_src.shape, ratings_trg.shape

(torch.Size([7, 97]),
 torch.Size([7, 5]),
 torch.Size([7, 97]),
 torch.Size([7, 5]))

In [28]:
(torch.rand((97, 7, 256)) * torch.rand((97, 7)).unsqueeze(-1)).shape

torch.Size([97, 7, 256])

In [29]:
torch.rand((97, 7)).unsqueeze(-1).shape

torch.Size([97, 7, 1])

In [30]:
PAD_IDX = pad_token#TRG.vocab.stoi['<pad>']

In [31]:
def RMSE_loss(prediction, target):
    return torch.sqrt(nn.MSELoss()(prediction, target))

In [32]:
def init_weights(m):
    # <YOUR CODE HERE>
    for name, param in m.named_parameters():
        nn.init.uniform_(param, -0.08, 0.08)
        


In [33]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)



In [34]:
def train(model, iterator, optimizer, criterion, clip, train_history=None, valid_history=None):
    model.train()
    
    epoch_loss = 0
    history = []
    for i, (tokens_src, tokens_trg, ratings_src, ratings_trg) in enumerate(tqdm(iterator)):
        
        optimizer.zero_grad()
        
        #output = model(src, trg)
        output, hidden = model(tokens_src.T, tokens_trg.T, ratings_src.T, ratings_trg.T)
        
        
        #trg = [trg sent len, batch size]
        #output = [trg sent len, batch size, output dim]
        
        output = output[1:].view(-1, output.shape[-1])
        tokens_trg = tokens_trg.T[1:].contiguous().view(-1)
        
        #trg = [(trg sent len - 1) * batch size]
        #output = [(trg sent len - 1) * batch size, output dim]
        
        loss1 = criterion(output, tokens_trg)
        loss2 = RMSE_loss(hidden, ratings_trg)        
        loss = 0.0*loss1 + loss2
        loss.backward()
        
        # Let's clip the gradient
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.cpu().data.numpy()
        
        history.append(loss.cpu().data.numpy())
        #if (i+1)%10==0:
        #    fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12, 8))

        #    clear_output(True)
        #    ax[0].plot(history, label='train loss')
        #    ax[0].set_xlabel('Batch')
        #    ax[0].set_title('Train loss')
        #    if train_history is not None:
        #        ax[1].plot(train_history, label='general train history')
        #        ax[1].set_xlabel('Epoch')
        #    if valid_history is not None:
        #        ax[1].plot(valid_history, label='general valid history')
        #    plt.legend()
            
        #    plt.show()

        
    return epoch_loss / len(iterator)

In [35]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    history = []
    
    with torch.no_grad():
    
        for i, (tokens_src, tokens_trg, ratings_src, ratings_trg) in enumerate(tqdm(iterator)):#

            #output = model(src, trg, 0) #turn off teacher forcing
            output, hidden = model(tokens_src.T, tokens_trg.T, ratings_src.T, ratings_trg.T)
            
            #trg = [trg sent len, batch size]
            #output = [trg sent len, batch size, output dim]

            output = output[1:].view(-1, output.shape[-1])
            trg = tokens_trg.T[1:].contiguous().view(-1)

            #trg = [(trg sent len - 1) * batch size]
            #output = [(trg sent len - 1) * batch size, output dim]

            loss = criterion(output, trg)
            loss2 = RMSE_loss(hidden, ratings_trg)
            
            #epoch_loss += 0.1*loss.cpu().data.numpy()
            epoch_loss += loss2.cpu().data.numpy()
        
    return epoch_loss / len(iterator)

In [36]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [37]:
def start_train(model, optimizer, criterion, n_epochs, train_history, valid_history):
    CLIP = 1
    best_valid_loss = float('inf')
    
    
    for epoch in tqdm(range(n_epochs)):

        start_time = time.time()

        #todo ds_test change to ds_train
        train_loss = train(model, DataLoader(ds_test, batch_size= 512), optimizer, criterion, CLIP, train_history, valid_history)
        valid_loss = evaluate(model, DataLoader(ds_test, batch_size= 512), criterion)

        end_time = time.time()

        epoch_mins, epoch_secs = epoch_time(start_time, end_time)

        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            torch.save(model.state_dict(), 'tut1-model.pt')

        train_history.append(train_loss)
        valid_history.append(valid_loss)
        print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s |\tTrain Loss: {train_loss:.3f} |\t Val. Loss: {valid_loss:.3f} ')
        
    
    plt.plot(train_history, label='general train history')
    plt.plot(valid_history, label='general valid history')
    plt.xlabel('Epoch')
    plt.legend()
    
    plt.show()

__Let's take a look at our network quality__:

In [38]:
N_EPOCHS = 10

In [39]:
#baseline

In [40]:
df_train, df_test = train_test_split(df_gr.loc[:1000], test_size=0.2, random_state=42)
len(df_train), len(df_test)

(800, 201)

In [41]:
ds_train = FeaturesDataset(df_train)
ds_test = FeaturesDataset(df_test)

## Эксперимент 3
Трафнсформер

In [42]:
from torch import Tensor
from torch.nn import Transformer
from timeit import default_timer as timer

In [43]:
#https://pytorch.org/tutorials/beginner/translation_transformer.html

# helper Module that adds positional encoding to the token embedding to introduce a notion of word order.
class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

# helper Module to convert tensor of input indices into corresponding tensor of token embeddings
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

# Seq2Seq Network 
class Seq2SeqTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        self.transformer = Transformer(d_model=emb_size,
                                       nhead=nhead,
                                       num_encoder_layers=num_encoder_layers,
                                       num_decoder_layers=num_decoder_layers,
                                       dim_feedforward=dim_feedforward,
                                       dropout=dropout)
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(
            emb_size, dropout=dropout)

    def forward(self,
                src: Tensor,
                trg: Tensor,
                src_mask: Tensor,
                tgt_mask: Tensor,
                src_padding_mask: Tensor,
                tgt_padding_mask: Tensor,
                memory_key_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None, 
                                src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer.encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer.decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)
    
    
    


SRC_VOCAB_SIZE = 105
TGT_VOCAB_SIZE = 105
EMB_SIZE = 512
NHEAD = 8
FFN_HID_DIM = 512
BATCH_SIZE = 128
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3

model = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE, 
                                 NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM)

for p in model.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

model = model.to(device)

loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)


In [44]:
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=device)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

In [45]:

def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=device).type(torch.bool)

    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

In [46]:

def train_epoch(model, optimizer, train_dataloader):
    model.train()
    losses = 0
  
    for batch in train_dataloader:
        
        src = batch.src
        tgt = batch.trg
        
        src = src.to(device)
        tgt = tgt.to(device)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

        optimizer.zero_grad()

        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        loss.backward()

        optimizer.step()
        losses += loss.item()

    return losses / len(train_dataloader)


def evaluate(model, val_dataloader):
    model.eval()
    losses = 0

    for batch in val_dataloader:
        
        src = batch.src
        tgt = batch.trg
        
        src = src.to(device)
        tgt = tgt.to(device)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)
        
        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        losses += loss.item()

    return losses / len(val_dataloader)

In [48]:

for epoch in range(1, N_EPOCHS+1):
    start_time = timer()
    train_loss = train_epoch(model, optimizer, DataLoader(ds_test, batch_size= 512))
    end_time = timer()
    val_loss = evaluate(model, valid_iterator)
    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "f"Epoch time = {(end_time - start_time):.3f}s"))



AttributeError: 'list' object has no attribute 'src'

In [ ]:

# function to generate output sequence using greedy algorithm 
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    src = src.to(device)
    src_mask = src_mask.to(device)

    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(device)
    for i in range(max_len-1):
        memory = memory.to(device)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                    .type(torch.bool)).to(device)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.item()

        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == EOS_IDX:
            break
    return ys


In [ ]:
EOS_IDX = SRC.vocab.stoi["<eos>"]
BOS_IDX = SRC.vocab.stoi["<sos>"]
EOS_IDX, BOS_IDX

In [ ]:
def get_text(x, TRG_vocab):
    text = []
    for token in x:
        try:
            text.append(TRG_vocab.itos[token])
        except Exception as ex:
            text.append('-')
    try:
        end_idx = text.index('<eos>')
        text = text[:end_idx]
    except ValueError:
        pass
    text = remove_tech_tokens(text)
    if len(text) < 1:
        text = []
    return text

In [ ]:
# actual function to translate input sentence into target language
def translate3(model: torch.nn.Module, src):
    model.eval()
    num_tokens = src.shape[0]
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    tgt_tokens = greedy_decode(
        model,  src, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX).flatten()

    res = get_text(tgt_tokens.cpu().numpy(), TRG.vocab)
    res = [x for x in res if x not in ["<sos>", "<eos>"]]
    return res

In [ ]:
original_text_trns = []
generated_text_trns = []

model.eval()
with torch.no_grad():

    for i, batch in tqdm.tqdm(enumerate(test_iterator)):
        
        src = batch.src
        tgt = batch.trg
        
        src = src.to(device)
        tgt = tgt.to(device)

        #tgt_input = tgt[:-1, :]

        #src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        #logits = transformer(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)
        
        #print(logits.shape)
        
        #print(logits.reshape(-1, logits.shape[-1]).shape)
        
        
        #tgt_out = tgt[1:, :]
        #print(tgt_out.reshape(-1).shape)
        
        #print(loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1)).item())
        
        original_text_trns.extend([get_text(x, TRG.vocab) for x in tgt.cpu().numpy().T])
        
        #print(tgt.shape)
        original = [get_text(x, TRG.vocab) for x in tgt.cpu().numpy().T]
        
        for xx in src.cpu().numpy().T:
            generated_text_trns.append(translate3(model, torch.tensor(xx).view(-1, 1)))
        
        #generated = [get_text(x, TRG.vocab) for x in tgt_tokens.cpu().numpy().T]
        
        
        

In [ ]:
score_trns = corpus_bleu([[text] for text in original_text_trns], generated_text_trns) * 100
score_trns

При использовании модели трансформера качество увеличилось, метрика BLEU выросла до 31

In [ ]:
models = ['baseline', 'baseline 2', 'attention', 'transformer']
scores = [score_baseline, score_1, score_att, score_trns]
original = [original_text_baseline, original_text_1, original_text_2, original_text_att, original_text_trns]
generated = [generated_text_baseline, generated_text_1, generated_text_2, generated_text_att, generated_text_trns]

for i in range(0, 7500, 250):
    for score, model_name, orig, gen in zip(scores, models, original, generated):
        print('model:', model_name, 'score:', round(score, 2))    
        print(' '.join(orig[i]), '->\n', ' '.join(gen[i]), '\n', sep='')
    print('\n\n')

Метрики моделей с вниманием и с трансформерами совпадают, но кажется что перевод модели с трансформером более качественный, это особенно заметно на длинных предложениях